# Fasting Pandas - A guide into optimizing your analytical processing 

## Lesson 2 - Data Parsing

---

Now that we know how to apply calculations more efficiently, it's time to put our skills to the test and do some serious data analysis. Once we are done we can share our analysis with the world by saving it into a file. Maybe we want to use that file later on to do some other work. Just another day at the office, or your home. Everyone works remote now, right?

Nooo, not yet. We need to take a step back and look into data parsing first.

Data parsing is the process of converting data from one format to another, and it's a crucial step in data analysis. In pandas, data parsing refers to the conversion of text data into a structured format, such as a DataFrame. This process involves not only reading data into memory but also converting it into appropriate data types.

Understanding data types is important because it can greatly impact the memory consumption of your program. By correctly identifying and assigning data types to each column, we can reduce the amount of memory needed to store the data, resulting in faster processing times and the ability to work with larger datasets. On the other hand, incorrect or inconsistent data types can lead to unexpected errors, increased memory usage, and slower processing times.

Different data types have different operations that can be performed on them, and using the appropriate data type can make certain operations faster and more efficient. For example, using a datetime data type instead of a string to represent dates can make date arithmetic operations more efficient. Maybe you have a column with numerical values that is stored as a string; by converting it to a numeric data type it can significantly reduce the amount of memory used.

In [1]:
import config
import os
import fasting_pandas as fp
import pandas as pd

### About this lesson

Our skills have grown and now we are vectorizing most of our calculations. They run quickly but we hear our computer screaming in agony. It seems reality caught up with us and we still need to learn a few more tricks before working with bigger datasets. Our computer's RAM is starting to melt and we need answers, fast.

We will be working with a new dataset called **team_results**. It consists of game results for various teams. The columns and their respective types and ranges are:

- size: categorical variable with three possible values: 'big', 'medium', and 'small'
- age: numerical variable representing the age of the player in years, with a range from 1 to 50.
- team: categorical variable representing the team name, with five possible values: 'yellow', 'cyan', 'magenta', 'violet', and 'black'.
- date: date variable representing the date of the game played, with a range from 2015-01-01 to 2020-12-31.
- prob: numerical variable representing the probability of winning the game, with values ranging from 0 to 1.
- result: categorical variable representing the result of the game, with two possible values: 'win' and 'lose'.

Our focus for this lesson will be based on the impact that the data types have in our computer's memory. Let's create a dataset of 10 million rows, way beyond the Excel realm and analyze what happens step by step.

In [2]:
df = fp.generate_teamresult_df(10_000_000)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 6 columns):
 #   Column  Dtype         
---  ------  -----         
 0   size    object        
 1   age     int32         
 2   team    object        
 3   date    datetime64[ns]
 4   prob    float64       
 5   result  object        
dtypes: datetime64[ns](1), float64(1), int32(1), object(3)
memory usage: 419.6+ MB


Just by parsing this dataset into a dataframe takes up 420mb of memory, which may not sound like much, but once you start doing transformations or merging it with other datasets, the memory usage can quickly add up. And what if you were dealing with even larger datasets, say 20, 50, or even 100 million rows? That's a lot of memory that you would need just to work with the data.

So, what can you do about it? Well, first you need to consider what you actually need from the data. Do you need all the columns? If not, you can filter out the unnecessary ones, which can make your program more memory-efficient.

Think of it this way: reading a whole dataset without selecting the specific columns you need is like using the ``SELECT *``  statement in SQL. It's not efficient, and can lead to unnecessary memory usage. So, it's important to be mindful of the data you're working with and only select the columns you need.

In our example, we'll save the dataset as a CSV and parse it back in as a DataFrame, selecting only the necessary columns. This will help us optimize our memory usage and make our analysis more efficient.

In [3]:
df.to_csv(os.path.join(config.DATA_DIR,'bad.csv'), index= False)
df = pd.read_csv(os.path.join(config.DATA_DIR,'bad.csv'), usecols = ['size', 'age', 'date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   size    object
 1   age     int64 
 2   date    object
dtypes: int64(1), object(2)
memory usage: 228.9+ MB


## The Impact of Data Types on Memory Usage

Something quite interesting has happened. Can you spot it?

First the memory usage decreased by almost half when we removed half the columns from the DataFrame. But the really interesting aspect is the Age data type **changed from int32 to int64**. In summary pandas automatically converts data types to their string representation when saving a DataFrame as a CSV file, and it uses type inference to determine the data types of each column when reading the file back in. I'll go over the mechanics to explain better.

When using the `to_csv()` method to save a DataFrame as a CSV file, pandas will convert the data types of the DataFrame to their string representation before writing the data to the CSV file. This means that all data types will be stored as strings in the CSV file, regardless of their original data type in the DataFrame.

For example, if a column in the DataFrame contains integer values, the `to_csv()` method will convert these integers to strings before writing them to the CSV file. Similarly, if a column contains dates or times, pandas will convert them to their string representation (e.g. '2023-03-28' for a date) before writing them to the CSV file. This behaviour can sometime lead to unexpected results.

When you call `read_csv()`, pandas will automatically try to infer the data types for each column based on the values it sees in the file. This is done using a process called type inference, which involves scanning through the first few rows of the file to determine the data types for each column.

What we presenced was the automatic conversion of integer data types from their original data type (e.g. int32) to a larger integer data type (e.g. int64) during the data type inference process. This can happen if the CSV file contains a large integer value that exceeds the range of the original integer data type, causing pandas to automatically upcast the data type to the next larger integer data type that can accommodate the largest value in the file.

### Downcasting Numbers to Save Memory

The takeaway is that we should be aware of the data we are dealing with, and based on it's extremes we can assign appropiately the size of the data type field in order to save memory. This is an important part of working with data in pandas, and understanding how it works can help you to effectively manipulate and analyze your data.

A handy dandy table of reference for you.

| Data Type | Range | Memory Consumption | Common Name |
|---|---|---|---|
| int8 | -128 to 127 | 1 byte | Byte |
| int16 | -32,768 to 32,767 | 2 bytes | Short |
| int32 | -2,147,483,648 to 2,147,483,647 | 4 bytes | Int |
| int64 |-9,223,372,036,854,775,808 to 9,223,372,036,854,775,807 | 8 bytes | Long |
| float16 | +/- 6.10 x 10^-5 to +/- 6.55 x 10^4, precision: 3 bits | 2 bytes | Half |
| float32 | +/- 1.18 x 10^-38 to +/- 3.40 x 10^38, precision: 7 digits | 4 bytes | Float |
| float64 | +/- 2.23 x 10^-308 to +/- 1.80 x 10^308, precision: 15 digits | 8 bytes | Double |

Note that the actual memory consumption may vary depending on the size and structure of the data, and whether the data is stored in a DataFrame or Series. Also, the ranges shown here are for the entire data type and may not reflect the actual range of values in a specific dataset. 

There is also a float128 data type, which is a quadruple-precision floating-point number with 128 bits. However, it's not a built-in data type in most programming languages and is generally only available through specialized libraries or hardware support. In pandas, the float128 data type is not supported.

Enough talk, lets see the magic happen.

In [4]:
df = pd.read_csv(os.path.join(config.DATA_DIR,'bad.csv'), usecols = ['size', 'age', 'date'],dtype={'age': 'int32'})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   size    object
 1   age     int32 
 2   date    object
dtypes: int32(1), object(2)
memory usage: 190.7+ MB


Cool. So we could have saved 30mb just by being aware of the change and reverting to the previous data type. But lets go a little bit deeper.

### Understanding the nature of your data

It seems Age is an interesting category to parse as an int8 data type in a DataFrame because no one should have negative ages or ages greater than 127 (unless we are dealing with robots or other non-human entities that might have an age beyond that range). By using an int8 data type to represent age, we can reduce the amount of memory used to store age values in the DataFrame.

To demonstrate this, let's consider a DataFrame with a single column representing age for a population of one million people. If we represent age as an int64 data type, which is the default data type for integer columns in pandas, each age value will occupy 8 bytes of memory. Therefore, the total memory used to store the age column in the DataFrame would be 8 MB (8 bytes * 1,000,000 values).

If we instead represent age as an int8 data type, each age value will occupy only 1 byte of memory. Therefore, the total memory used to store the age column in the DataFrame would be 1 MB (1 byte * 1,000,000 values), which is 8 times smaller than when using int64. This represents a significant reduction in memory usage, especially for large datasets.

However, there is a trade-off between memory usage and data precision. Using an int8 data type to represent age means that we can only represent ages with a maximum range of 127. If we have any ages outside this range, they will be truncated or result in an error. Therefore, we need to carefully consider the data range of the age column when choosing the appropriate data type.

We can also consider using int16 or int32 data types to represent age, which would allow us to represent a wider range of ages while still reducing the memory usage compared to using int64. However, these data types would still require more memory than int8 and should only be used if the age values fall outside the range of -128 to 127.

In summary, choosing an appropriate data type for age in a DataFrame can help reduce memory usage while ensuring data precision. For now it seems reasonable to work with an int8 data type for Age as it can significantly reduce memory. Lets do a quick demostration with pandas.

In [5]:
print(f"Parsing Age as an int8 consumes { df['age'].astype('int8').memory_usage() / 1024 ** 2:.2f} MB of memory")
print(f"Parsing Age as an int16 consumes { df['age'].astype('int16').memory_usage() / 1024 ** 2:.2f} MB of memory")
print(f"Parsing Age as an int32 consumes { df['age'].astype('int32').memory_usage() / 1024 ** 2:.2f} MB of memory")
print(f"Parsing Age as an int64 consumes { df['age'].astype('int64').memory_usage() / 1024 ** 2:.2f} MB of memory")

Parsing Age as an int8 consumes 9.54 MB of memory
Parsing Age as an int16 consumes 19.07 MB of memory
Parsing Age as an int32 consumes 38.15 MB of memory
Parsing Age as an int64 consumes 76.29 MB of memory


From 9.5 MB to 76.3 MB. Our theory holds true since that is an 8x difference of unnecesary memory waste.

Let's look at our other columns: size and date. Currently they are parsed as objects, which is default behaviour. But what are objects? We need to understand them before learning to change them.

## Object Data Type

In Python, an object is a collection of data and functions that work together to perform a specific task. The memory usage of an object depends on several factors such as the size of the data it stores, the number of methods and attributes it has, and the type of data it contains.

Objects are memory intensive. It means that it consumes a significant amount of memory compared to other data types. This is because objects can be dynamically created and can contain arbitrary data types, including other objects, which can lead to a larger memory footprint.

For example, in pandas, an object column can contain strings of varying lengths, which can lead to significant memory usage. This is because each string is stored as a separate object, and the size of each object can vary depending on the length of the string. We can appreciate this behaviour when analyzing the object type with ``df.info()`` because it will have a + sign suffix on the memory usage, indicating that it might vary based on the length and amount of inputs.

We can break it down the following way to check how much bytes each record consume based on its memory and the total amount. We will also compare it with the information given by ``df.info()`` and see if it checks out. This section is code intensive so I tried to be as explicit as possible so you can follow along.

I'll make use of the regex and io libraries so we can also do some cool string parsing.

In [6]:
import io
import re
# Get information about size column and store the output into a StringIO buffer.
buffer = io.StringIO()
df['size'].info(buf=buffer)
# Retrieve the contents of the StringIO buffer and split it into a list of lines
info_list = buffer.getvalue().splitlines()
# Search the last line of the info_list for a match to the pattern and extract the first match
pattern = re.compile(r"memory usage: (\d+\.\d+)")
info_memory_usage_str = re.findall(pattern, info_list[-1])[0]
info_memory_usage = float(info_memory_usage_str)

In [7]:
# Get information about the object memory consumption through memory_usage
memory_usage_column = df['size'].memory_usage() / 1024**2
memory_usage_total = df['size'].memory_usage(deep= True) / 1024**2

In [8]:
# Generate information about the memory usage of each value in the column by using sys.getsizeof
import sys
import numpy as np
memory_breakdown_df = pd.DataFrame({
    'record_length':df['size'].map(len).unique(), 
    'memory_per_record': np.array([sys.getsizeof(val) for val in df['size'].unique()]), 
    'total_records':df['size'].value_counts()
    }).reset_index(names = 'record')
memory_breakdown_df['total_memory'] = memory_breakdown_df['memory_per_record'] * memory_breakdown_df['total_records']
memory_usage_records = memory_breakdown_df.total_memory.sum() / 1024**2

In [13]:
print(f'The memory displayed by df.info() is {info_memory_usage:.2f} MB+')
print(f'The memory consumed by the object column is {memory_usage_column:.2f} MB')
print('Record breakdown information:')
print(memory_breakdown_df)
print(f'The memory consumed by the records in the column is: {memory_usage_records:.2f} MB')
print(f'The total memory of the DataFrame object is {memory_usage_total:.2f} MB ')
print(f'Is the sum of the memory consumed by the objects column + the records is the same as memory_usage(deep=True)? {"Yes." if round(memory_usage_records + memory_usage_column,1) == round(memory_usage_total,1) else "No!"}')

The memory displayed by df.info() is 76.30 MB+
The memory consumed by the object column is 76.29 MB
Record breakdown information:
   record  record_length  memory_per_record  total_records  total_memory
0     big              3                 52        3334794     173409288
1  medium              6                 55        3333373     183335515
2   small              5                 54        3331833     179918982
The memory consumed by the records in the column is: 511.80 MB
The total memory of the DataFrame object is 588.10 MB 
Is the sum of the memory consumed by the objects column + the records is the same as memory_usage(deep=True)? Yes.


Interesting! So we have been misguided this whole time by ``df.info()`` when calculating object types, as it was only accounting the memory used by the column (pointer) itself, not the memory usage of the objects that are stored in the column (the data structure). This means that if the column contains objects, ``df.info()`` and ``memory_usage()`` will only return the memory usage of the pointers to those objects, not the memory usage of the objects themselves.

Therefore, the memory consumption of an object is the sum of the memory consumed by the reference plus the memory consumed by the data it refers to.

On the other hand, ``memory_usage(deep=True)`` calculates the memory usage of the entire dataframe, taking into account the memory usage of the objects stored in each column. This means that if the column contains objects, ``memory_usage(deep=True)`` will return the memory usage of the objects themselves, not just the pointers to those objects.

This should emphasize the importance of being aware of how we store objects in general and how to accurately calculate their memory usage. It seems we are wasting a lot of memory on objects, so how can we do better?

## Categorical Data Type

Enter the categorical type. In essence, a category is a fixed number of possible and limited values. Categorical data can be represented more efficiently in memory as compared to object or string data types.

When a column is parsed as a categorical data type, pandas stores the column values as integers and maintains a separate dictionary to map these integer values back to the original categories. This allows pandas to save memory by representing the same categories with fewer bytes.

For example, let's say we have a column in a DataFrame that represents the country of origin of a product, and there are only 10 unique countries in the column. If we represent this column as an object data type, each value will take up a lot of memory, especially if the column contains long country names or strings. However, if we represent this column as a categorical data type, pandas will only store the integer values and a dictionary mapping these integers back to the original country names. This can result in significant memory savings, especially for large datasets with many unique values.

Another benefit of using the categorical data type is faster computation. Categorical columns are typically faster to sort, group, and aggregate compared to object or string columns, as the computation is performed on integers rather than long strings.

But..

Categorical columns are quite fragile and complex. One issue with categorical columns is that they are designed to work with a specific set of operations. For example, if you try to perform string operations on a categorical column, you may get unexpected results or errors. Concatenating a categorical column with a string is not possible because they are fundamentally different data types. As stated previously, categorical data is stored as numerical codes and strings are stored as a sequence of characters. If you try to merge them pandas will raise a TypeError because it doesn't know how to combine the two different data types.

Another possible drawback is that the categorical data type may not be suitable for columns with a large number of unique values. In such cases, the memory savings achieved by using the categorical data type may be minimal, or even worse, since the overhead of maintaining the category dictionary may actually increase memory usage and slow down computation.

You will also find you may not be able to perform certain mathematical operations like summing the values of a categorical column unless you first convert it to a different data type. Performing the computation will result in a memory spike as all benefits were lost. This is because categorical data is typically treated as discrete, rather than continuous. What you can do is aggregating them by using a variety of methods, such as:

- Count: Counting the number of occurrences of each category in the column
- Unique: Counting the number of unique categories in the column
- Mode: Identifying the most frequently occurring category in the column
- Sum: Aggregating a numeric column by the categories in a categorical column

So while categorical columns can be faster for certain types of operations and potentially save a ton of memory, they are not always suitable for all types of data or all types of operations. It's important to carefully consider the type of data you are working with and the types of operations you need to perform in order to determine whether categorical data is appropriate and how to handle it properly.

Having said that, I will show you now the difference when parsing a column as an object or category.

In [10]:
date_category_memory = df.date.astype('category').memory_usage() / 1024 ** 2
date_object_memory = df.date.astype('object').memory_usage(deep = True) / 1024 ** 2
date_datetime_memory = df.date.astype('datetime64[ns]').memory_usage() / 1024 ** 2

print(f'Parsing date as a category consumes {date_category_memory:.2f} MB')
print(f'Parsing date as an object consumes {date_object_memory:.2f} MB')
print(f'Parsing date as a datetime consumes {date_datetime_memory:.2f} MB')

print(f' Between object and category is {fp.calculate_percentage_difference(date_object_memory, date_category_memory):.2f}%')
print(f' Between object and datetime is {fp.calculate_percentage_difference(date_object_memory, date_datetime_memory):.2f}%')

Parsing date as a category consumes 19.15 MB
Parsing date as an object consumes 638.96 MB
Parsing date as a datetime consumes 76.29 MB
--- Memory consumption difference ---
 Between object and category is -97.00%
--- Memory consumption difference ---
 Between object and datetime is -88.06%


Significant, to say the least. 

## Datetime Data Type

Notice how the datetime data type stores memory much more efficiently than the object type. If you look closer, they use the same amount of memory as the object's pointers. This is because datetime objects are represented as a fixed-size format in memory, regardless of the size of the input data. Object types on the other hand vary depending on the underlying data, as we saw earlier. 

A pandas datetime object is typically represented as a 64-bit integer representing the number of nanoseconds since January 1, 1970, at 00:00:00 UTC. This is the same as the Unix epoch. This means that datetime objects only take up 8 bytes of memory (64 bits) to store the date and time information.

Additionally, datetime objects in pandas are typically stored as a separate array that is aligned with the index of the DataFrame. This means that the memory consumption of the datetime column is largely independent of the memory consumption of the rest of the DataFrame.

Therefore, if you need to work with date and time information in your code, it is recommended to use datetime data types. This allows for faster operations such as comparisons and arithmetic calculations. Additionally, datetime data types have built-in methods for formatting and parsing dates and times, making them easier to work with in code.

Putting it all together.

In [11]:
# Read
terrible_df = pd.read_csv(os.path.join(config.DATA_DIR,'bad.csv'))
better_df = pd.read_csv(os.path.join(config.DATA_DIR,'bad.csv'), usecols=['size', 'age', 'date'], dtype={'age': 'int8', 'size': 'category'}, parse_dates=['date'])
# Compare.
terrible_df_memory = terrible_df.memory_usage(deep=True).sum() / 1024 **2
better_df_memory = better_df.memory_usage(deep=True).sum() / 1024 **2
print('\nComparing memory consumption by dataframe')
print(f'The (real) total amount of memory consumed by terrible_df is {terrible_df_memory:.2f} MB')
print(f'The (real) total amount of memory consumed by better_df is {better_df_memory:.2f} MB')
print(f'terrible_df vs better_df: {fp.calculate_percentage_difference(terrible_df_memory, better_df_memory):.2f}%')
print('\nComparing memory consumption by column')
print('terrible_df')
terrible_df_columns_memory = fp.calculate_memory_usage(terrible_df[['size', 'age', 'date']])
print('better_df')
better_df_columns_memory = fp.calculate_memory_usage(better_df)
print(fp.calculate_percentage_difference(terrible_df_columns_memory, better_df_columns_memory))


Comparing memory consumption by dataframe
The (real) total amount of memory consumed by terrible_df is 2553.62 MB
The (real) total amount of memory consumed by better_df is 95.37 MB
--- Memory consumption difference ---
terrible_df vs better_df: -96.27%

Comparing memory consumption by column
terrible_df
--- Memory consumption ---
Index          128
size     616663785
age       80000000
date     670000000
dtype: int64

better_df
--- Memory consumption ---
Index         128
size     10000185
age      10000000
date     80000000
dtype: int64

--- Memory consumption difference ---
Index     0.000000
size    -98.378341
age     -87.500000
date    -88.059701
dtype: float64


By incorporating best practices into our workflow we reduced our dataframe memory consumption from 2.5 GB to 95 MB. So much room for possibilities!

Note that I parsed date as a datetime object instead of a categorical object, even though I could have saved more memory. I avoided so because it didn't make practical sense for my hipotetical purposes, since I want to perform some analysis based on time and having date as a category would limit my options. Our main purpose is not to make our DataFrame as efficient as possible, rather make it as practical as possible in the most efficient manner. That is the key takeaway of this lesson.

I'll delete the csv file as we don't need it anymore.

In [12]:
try:
    os.remove(os.path.join(config.DATA_DIR, 'bad.csv'))
except FileNotFoundError as e:
    print(e)

## Concluding thoughts

Data parsing is a crucial step in data analysis that can significantly impact the accuracy and efficiency of downstream tasks. Accurately parsing data is crucial to ensure that it is properly represented because it can save time, resources and reduce errors in data analysis workflows. This will allow you to focus on the insights and value that your data can provide.

However, despite the importance of accurate data parsing, many data analysts still work with CSV files, which are notorious for being inefficient and difficult to parse. CSV files often have inconsistent formats, missing values, and require manual handling of data types.

Not to worry. I will show you the way. Nobody will be able to say we trained you, as a joke.

Go to [Lesson 3 - Read and Write](3-read-and-write.ipynb)